In [1]:
# toggle to save space
mode = '_mean' if False else '_all'
print(mode)

_all


In [2]:
gbm_params = {
    'n_estimators' : 50, # 500,  
    'max_depth' : 3,
    'learning_rate': 0.9,
    'bagging_fraction': 0.8, # TODO: try 0.9
    
    'feature_fraction' : 0.9,
    'bagging_freq': 5,
    'subsample' : 0.1,  # 
    'subsample_freq' : 1,
    'num_leaves' : 20,
    'metric':'rmse',
    #'lambda_l1' : 1,  # Try defaults
    #'lambda_l2': 1, # Try defaults
    'verbose': 100
}

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from tqdm import tqdm
from datetime import date 
import holidays
import lightgbm as lgb


warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [4]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [5]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.float32, 'year_built': np.float16, 'floor_count': np.float16},
}

def loadFile(name):
    for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
        if path.exists(dir_path + name + '.csv'):
            return  ConvertToDatetime().transform(
                pd.read_csv(dir_path + name + '.csv', dtype=file_dtype[name]))
        


In [6]:
building = loadFile('building_metadata')
pre_train = loadFile('train')
#test = loadFile('test')

weather_processed_df = pd.read_pickle(f'../input/ashrae-energy-prediction-pickles/weather_processed{mode}.pickle')

In [7]:
def merge(x):
    weather_processed_df = pd.read_pickle(f'../input/ashrae-energy-prediction-pickles/weather_processed{mode}.pickle')
    x = x.merge(building, on=['building_id'], how='left')
    gc.collect()
    x = x.merge(weather_processed_df, on=['site_id', 'timestamp'], how='left')
    gc.collect()
    x['site_id'] = x['site_id'].astype('int8');
    x['cloud_coverage'] = x['cloud_coverage'].astype('float16')
    gc.collect()
    return x
        
train = merge(pre_train)  
print(train)
print('!!!! Warning we are missing weather for '+ str(train['air_temperature'].isnull().sum())+' rows')
train = train.dropna(axis=0, subset=['air_temperature'])


          building_id  meter           timestamp  meter_reading  site_id  \
0                   0      0 2016-01-01 00:00:00       0.000000        0   
1                   1      0 2016-01-01 00:00:00       0.000000        0   
2                   2      0 2016-01-01 00:00:00       0.000000        0   
3                   3      0 2016-01-01 00:00:00       0.000000        0   
4                   4      0 2016-01-01 00:00:00       0.000000        0   
5                   5      0 2016-01-01 00:00:00       0.000000        0   
6                   6      0 2016-01-01 00:00:00       0.000000        0   
7                   7      0 2016-01-01 00:00:00       0.000000        0   
8                   8      0 2016-01-01 00:00:00       0.000000        0   
9                   9      0 2016-01-01 00:00:00       0.000000        0   
10                 10      0 2016-01-01 00:00:00       0.000000        0   
11                 11      0 2016-01-01 00:00:00       0.000000        0   
12          

[20216100 rows x 101 columns]
!!!! Warning we are missing weather for 89888 rows


In [8]:
# See holiday notebook to generate, this is optional
holiday_df = None
if path.exists('../input/ashrae-energy-prediction-pickles/holiday_df.pickle'):
    holiday_df = pd.read_pickle('../input/ashrae-energy-prediction-pickles/holiday_df.pickle')
if holiday_df is not None:
    print(holiday_df.sample(20))

          site_id           timestamp                         holiday
18137298       15 2016-11-24 18:00:00                    Thanksgiving
46650046       12 2018-01-01 18:00:00                  New Year's Day
946637         10 2016-01-18 04:00:00     Martin Luther King, Jr. Day
45564926       10 2017-10-09 00:00:00                    Columbus Day
13591078       14 2016-09-05 02:00:00                       Labor Day
19982313        1 2016-12-27 21:00:00        Christmas Day (Observed)
61808275       10 2017-01-01 00:00:00                  New Year's Day
36263260        5 2018-08-06 16:00:00  Summer Bank Holiday [Scotland]
18113687       15 2016-11-24 08:00:00                    Thanksgiving
8130216         9 2016-05-30 14:00:00                    Memorial Day
19843829        7 2016-12-25 10:00:00                   Christmas Day
45230076       10 2017-01-02 07:00:00       New Year's Day (Observed)
46644538       12 2017-12-26 09:00:00                      Boxing Day
35732642        5 20

In [9]:
class MeterReadingLog1p(TransformerMixin):
  
    def transform(self, df, **transform_params):
        if 'meter_reading' in df.columns:
            df['meter_reading_log1p'] = np.log1p(df['meter_reading'])
            df = df.drop('meter_reading', axis=1)
        return df
    
    def fit(self, X, y=None, **fit_params):
        return self
print(train.sample(20, random_state=42))
print(MeterReadingLog1p().transform(train.sample(20, random_state=42)))
gc.collect()

          building_id  meter           timestamp  meter_reading  site_id  \
19993926         1389      1 2016-12-28 01:00:00      18.774500       15   
6530527          1240      3 2016-05-02 03:00:00       0.000000       14   
6484853           464      0 2016-05-01 08:00:00      13.480000        3   
18072271          656      0 2016-11-23 15:00:00      11.100000        5   
15944342          322      0 2016-10-16 18:00:00      24.889999        3   
4563775           643      0 2016-03-27 23:00:00      72.724197        4   
18118279         1278      0 2016-11-24 10:00:00     113.069000       14   
2121957           575      0 2016-02-08 11:00:00      69.300003        4   
18890113          890      2 2016-12-08 14:00:00     631.349976        9   
14306902         1390      2 2016-09-17 18:00:00     134.160004       15   
5056620           702      0 2016-04-05 22:00:00       9.500000        5   
6530049           991      2 2016-05-02 03:00:00       0.000000        9   
5177920     

[20 rows x 101 columns]
          building_id  meter           timestamp  site_id  \
19993926         1389      1 2016-12-28 01:00:00       15   
6530527          1240      3 2016-05-02 03:00:00       14   
6484853           464      0 2016-05-01 08:00:00        3   
18072271          656      0 2016-11-23 15:00:00        5   
15944342          322      0 2016-10-16 18:00:00        3   
4563775           643      0 2016-03-27 23:00:00        4   
18118279         1278      0 2016-11-24 10:00:00       14   
2121957           575      0 2016-02-08 11:00:00        4   
18890113          890      2 2016-12-08 14:00:00        9   
14306902         1390      2 2016-09-17 18:00:00       15   
5056620           702      0 2016-04-05 22:00:00        5   
6530049           991      2 2016-05-02 03:00:00        9   
5177920           224      0 2016-04-08 02:00:00        2   
19974616         1247      0 2016-12-27 17:00:00       14   
9989434            43      1 2016-07-02 15:00:00        0   


[20 rows x 101 columns]


31

In [10]:
meter_desc_columns={'mean': 'meter_mean', 'max': 'meter_max', 'min': 'meter_min', 'std':'meter_std'}

class CreateMeterDescDF(TransformerMixin):

    def transform(self, df, **transform_params):
        global _building_meter_desc_DF
        print(df.columns)
        if 'meter_reading_log1p' in df.columns:
            cols = ['mean']
            if mode == '_all':
                cols = ['mean','max','min','std']
            group = df.groupby(['building_id','meter'])['meter_reading_log1p']
            desc_DF = group.describe()[cols]
            desc_DF = desc_DF.reset_index()
            _building_meter_desc_DF = desc_DF.rename(columns=meter_desc_columns)
            gc.collect()
        return df 
    def fit(self, X, y=None, **fit_params):
        return self

#if 'meter_mean' not in train.columns:
#    print(building_meter_desc_DF)
#    train = train.merge(building_meter_desc_DF, on=['building_id','meter'], how='left')
#    #test = test.merge(building_meter_desc_DF, on=['building_id','meter'], how='left')
#    del building_meter_desc_DF
CreateMeterDescDF().transform(MeterReadingLog1p().transform(train.sample(2000, random_state=0)))    
print(_building_meter_desc_DF)
gc.collect()

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage',
       ...
       'sea_level_pressure_std_lag72', 'wind_direction_mean_lag72',
       'wind_direction_max_lag72', 'wind_direction_min_lag72',
       'wind_direction_std_lag72', 'wind_speed_mean_lag72',
       'wind_speed_max_lag72', 'wind_speed_min_lag72', 'wind_speed_std_lag72',
       'meter_reading_log1p'],
      dtype='object', length=101)
      building_id  meter  meter_mean  meter_max  meter_min  meter_std
0               1      0    4.911728   4.911728   4.911728        NaN
1               2      0    1.907095   1.907095   1.907095        NaN
2               3      0    2.450559   4.901118   0.000000   3.465614
3               4      0    0.000000   0.000000   0.000000        NaN
4               5      0    0.000000   0.000000   0.000000        NaN
5               6      0    4.392713   4.392713   4.392713        NaN

21

In [11]:
class MergeMeterDescDF(TransformerMixin):
  
    def transform(self, df, **transform_params):
        # drop any columns to add
        df = df.drop(meter_desc_columns.values(), axis=1, errors='ignore') 
        return df.merge(_building_meter_desc_DF, on=['building_id','meter'], how='left')

    def fit(self, X, y=None, **fit_params):
        return self

print(MergeMeterDescDF().transform(train.sample(2000, random_state=0)))

      building_id  meter           timestamp  meter_reading  site_id  \
0             683      0 2016-08-16 06:00:00       1.700000        5   
1            1265      0 2016-02-17 20:00:00     142.848007       14   
2            1030      1 2016-11-24 18:00:00       0.000000       11   
3            1240      3 2016-12-25 11:00:00     413.479004       14   
4             857      0 2016-07-12 18:00:00       0.000000        8   
5              18      0 2016-08-16 21:00:00    1876.219971        0   
6             712      0 2016-07-17 01:00:00       1.900000        5   
7             416      0 2016-04-10 06:00:00      11.250000        3   
8             715      0 2016-08-17 23:00:00       4.500000        5   
9             183      1 2016-05-19 06:00:00     289.365997        2   
10            204      0 2016-09-10 01:00:00      34.680000        2   
11            249      1 2016-11-05 04:00:00     605.284973        2   
12            633      0 2016-05-13 18:00:00      27.610001     

[2000 rows x 105 columns]


In [12]:
# "As you can see above, this data looks weired until May 20. It is 
# reported in this discussion by @barnwellguy that All electricity
# meter is 0 until May 20 for site_id == 0. Let's remove these data 
# from training data."
# https://www.kaggle.com/kaushal2896/ashrae-eda-fe-lightgbm-1-13
class RmS0M0(TransformerMixin):
  
    def transform(self, df, **transform_params):
        return df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

    def fit(self, X, y=None, **fit_params):
        return self
    


In [13]:
# TODO: write filter to remove any 0 meter reading that continue more than N days (try 3)
# Also we need to account for this by meter

In [14]:
# TODO: try rolling with power

In [15]:
    
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114483#latest-660771
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114874#latest-660970
class AddHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        if holiday_df is not None:
            if 'holiday' in df.columns:
                df = df.drop('holiday', axis=1)
            df = df.merge(holiday_df, on=['timestamp','site_id'], how='left')
            df['holiday'] = df['holiday'].astype('category')
        else:
            print("Warning: Holiday DF is missing")
        return df

    def fit(self, X, y=None, **fit_params):
        return self
# Test 
if holiday_df is not None:
    print(holiday_df.columns)
    print(AddHolidays().transform(train.head(20))[['holiday','timestamp']])

Index(['site_id', 'timestamp', 'holiday'], dtype='object')
           holiday  timestamp
0   New Year's Day 2016-01-01
1   New Year's Day 2016-01-01
2   New Year's Day 2016-01-01
3   New Year's Day 2016-01-01
4   New Year's Day 2016-01-01
5   New Year's Day 2016-01-01
6   New Year's Day 2016-01-01
7   New Year's Day 2016-01-01
8   New Year's Day 2016-01-01
9   New Year's Day 2016-01-01
10  New Year's Day 2016-01-01
11  New Year's Day 2016-01-01
12  New Year's Day 2016-01-01
13  New Year's Day 2016-01-01
14  New Year's Day 2016-01-01
15  New Year's Day 2016-01-01
16  New Year's Day 2016-01-01
17  New Year's Day 2016-01-01
18  New Year's Day 2016-01-01
19  New Year's Day 2016-01-01


In [16]:
class RmHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        if holiday_df is not None:
            df = df.merge(holiday_df, on=['timestamp','site_id'], how='left')
            df = df.drop(df[df['holiday'].notnull()].index)
            df = df.drop(['holiday'], axis=1)
            gc.collect()
        else:
            print("Warning: Holiday DF is missing")
        return df

    def fit(self, X, y=None, **fit_params):
        return self

# Test you should see the new years removed
#print(train.head(100000).merge(building, on='building_id', how='left').columns)
print(RmHolidays().transform(train.head(100000)))

       building_id  meter           timestamp  meter_reading  site_id  \
52897            0      0 2016-01-02 00:00:00       0.000000        0   
52898            1      0 2016-01-02 00:00:00       0.000000        0   
52899            2      0 2016-01-02 00:00:00       0.000000        0   
52900            3      0 2016-01-02 00:00:00       0.000000        0   
52901            4      0 2016-01-02 00:00:00       0.000000        0   
52902            5      0 2016-01-02 00:00:00       0.000000        0   
52903            6      0 2016-01-02 00:00:00       0.000000        0   
52904            7      0 2016-01-02 00:00:00       0.000000        0   
52905            8      0 2016-01-02 00:00:00       0.000000        0   
52906            9      0 2016-01-02 00:00:00       0.000000        0   
52907           10      0 2016-01-02 00:00:00       0.000000        0   
52908           11      0 2016-01-02 00:00:00       0.000000        0   
52909           12      0 2016-01-02 00:00:00      

[43848 rows x 101 columns]


In [17]:
class SetCatTypes(TransformerMixin):
    
    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col]= df[col].astype('category')
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [18]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [19]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(building.head(20)['square_feet'])

0       7432.0
1       2720.0
2       5376.0
3      23685.0
4     116607.0
5       8000.0
6      27926.0
7     121074.0
8      60809.0
9      27000.0
10    370773.0
11     49073.0
12     37100.0
13     99380.0
14     86250.0
15     83957.0
16     54644.0
17     15250.0
18    111891.0
19     18717.0
Name: square_feet, dtype: float32


In [20]:
# TODO: Play with scaling cloud coverage

In [21]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        # revisit the choice of median vs anything else
        tmp_df = train.drop_duplicates(['site_id','building_id'])[['site_id','building_id','year_built']]
        year_built_median = tmp_df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in tmp_df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        del tmp_df, year_built_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(ImputeYearBuilt().transform(train.sample(20))['building_age'])

14679539     70
5331216      70
11328278     70
1378535      55
5602851      70
2143986      70
6058699      62
12541301     62
12139102     53
18674557     70
9703121      70
7814310      70
7620595      70
9489886      75
20082197     64
15794950     66
19214111     70
8451115      74
1599008     105
15956832     70
Name: building_age, dtype: uint8


In [22]:
class ImputeFloorCount(TransformerMixin):

    def transform(self, df, **transform_params):
        # revisit the choice of median vs anything else
        tmp_df = train.drop_duplicates(['site_id','building_id'])[['site_id','building_id','floor_count']]
        floors_median = tmp_df['floor_count'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in tmp_df.groupby(['site_id'])['floor_count'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['floor_count'].isnull()) & (df['site_id'] == i), 'floor_count'] = i_median
            else:
                df.loc[(df['floor_count'].isnull()) & (df['site_id'] == i), 'floor_count'] = floors_median
        del tmp_df, floors_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

print(ImputeFloorCount().transform(train.sample(20))['floor_count'])

2483891     3.0
15709539    8.0
9009247     5.0
16512376    3.0
5375110     3.0
18223335    6.0
10002046    3.0
8978018     3.0
15541701    3.0
4764192     2.0
2189666     3.0
16032597    3.0
3569352     3.0
7713558     2.0
18397218    3.0
9126649     3.0
16450992    4.0
4764035     3.0
6368104     2.0
13110559    3.0
Name: floor_count, dtype: float16


In [23]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [24]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        # TODO: try week of year as numerical 
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('category') # vs weekend?
        #df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['dayofweek_hour'] = (df['timestamp'].dt.dayofweek * 24) + df['timestamp'].dt.hour
        #df['dayofweek_hour'] = df['dayofweek_hour'].astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('uint8')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [25]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [26]:
class FillMean(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].mean())
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [27]:
class FillZeros(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(0)
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [28]:
class FillMedian(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].median())
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [29]:
class FillPopular(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].value_counts()[0])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [30]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [31]:
class GC(TransformerMixin):
        
    def transform(self, df, **transform_params):
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [32]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred.clip(0)))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)

def lbm_rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

# rob's custome function to do RMSLE while in the log1p space
def lbm_rmslee(y_true, y_pred):
    return 'RMSLEE', np.sqrt(np.mean(np.power(y_pred - y_true, 2))), False



In [33]:
%%time

x_pre_pipes = Pipeline(
    steps=[
        ('meterReadingLog1p',MeterReadingLog1p()),
        ('rmS0M0', RmS0M0()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('logSquareFeet', LogSquareFeet()),
        #('rmHolidays', RmHolidays()),
        #('addHolidays', AddHolidays()),
        ('createMeterDescDF', CreateMeterDescDF()), # note declares a globe variable to pass
        ('mergeMeterDescDF', MergeMeterDescDF()), # populates both test and train from global
        ('setCatTypes', SetCatTypes(['building_id', 'site_id', 'meter', 'primary_use'])),
        ('GC', GC())
    ]
)

train = x_pre_pipes.transform(train)
print(train.columns)

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage',
       ...
       'wind_direction_std_lag72', 'wind_speed_mean_lag72',
       'wind_speed_max_lag72', 'wind_speed_min_lag72', 'wind_speed_std_lag72',
       'meter_reading_log1p', 'dayofweek', 'hour', 'log_square_feet',
       'holiday'],
      dtype='object', length=105)
Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage',
       ...
       'wind_speed_std_lag72', 'meter_reading_log1p', 'dayofweek', 'hour',
       'log_square_feet', 'holiday', 'meter_mean', 'meter_max', 'meter_min',
       'meter_std'],
      dtype='object', length=109)
Wall time: 1min 11s


In [34]:
# pre_a_pipes is for preprocessing that doesn't change impute
# values
x_fold_pipes = Pipeline(
    steps=[
        #('markNans',MarkNaNs()),
        #('convertToDatetime', ConvertToDatetime()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('imputeFloorCount', ImputeFloorCount()),
        ('dropCols', DropCols(['timestamp','square_feet', 'year_built'])),
        ('GC', GC())
    ]
)

sample_train_X = x_fold_pipes.transform(train.sample(20))
print(sample_train_X.columns)
print(sample_train_X.dtypes)

Index(['building_id', 'meter', 'site_id', 'primary_use', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure',
       ...
       'meter_reading_log1p', 'dayofweek', 'hour', 'log_square_feet',
       'holiday', 'meter_mean', 'meter_max', 'meter_min', 'meter_std',
       'building_age'],
      dtype='object', length=107)
building_id                      category
meter                            category
site_id                          category
primary_use                      category
floor_count                       float16
air_temperature                   float16
cloud_coverage                    float16
dew_temperature                   float16
precip_depth_1_hr                 float16
sea_level_pressure                float16
wind_direction                    float16
wind_speed                        float16
s_radiation                       float64
air_temperature_mean_lag3         float16
air_temperature_m

In [35]:
# this stratified strategy from
# https://www.kaggle.com/isaienkov/lightgbm-fe-1-19/notebook
folds = 5
kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=42)

In [36]:
models = []
best_scores = []
for train_index, val_index in kf.split(train, train['building_id']):
    f_train_X = x_fold_pipes.transform(train.loc[train_index])
    f_val_X = x_fold_pipes.transform(train.loc[val_index])
    gbm = LGBMRegressor(**gbm_params)
    gbm.fit(f_train_X.drop('meter_reading_log1p', axis=1), f_train_X['meter_reading_log1p'],
        eval_set=[(f_val_X.drop('meter_reading_log1p', axis=1), f_val_X['meter_reading_log1p'])],
        # https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114722#latest-660848
        # eval_metric=lbm_rmslee,
        early_stopping_rounds=20)
    models.append(gbm)
    best_scores.append(gbm.best_score_)
    del f_train_X, f_val_X, gbm
    gc.collect()


[1]	valid_0's rmse: 1.3482
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.25624
[3]	valid_0's rmse: 1.26028
[4]	valid_0's rmse: 1.26284
[5]	valid_0's rmse: 1.26012
[6]	valid_0's rmse: 1.25829
[7]	valid_0's rmse: 1.26165
[8]	valid_0's rmse: 1.26226
[9]	valid_0's rmse: 1.24865
[10]	valid_0's rmse: 1.1808
[11]	valid_0's rmse: 1.18183
[12]	valid_0's rmse: 1.17941
[13]	valid_0's rmse: 1.17984
[14]	valid_0's rmse: 1.17445
[15]	valid_0's rmse: 1.1744
[16]	valid_0's rmse: 1.1725
[17]	valid_0's rmse: 1.16169
[18]	valid_0's rmse: 1.16346
[19]	valid_0's rmse: 1.16504
[20]	valid_0's rmse: 1.16463
[21]	valid_0's rmse: 1.16619
[22]	valid_0's rmse: 1.16705
[23]	valid_0's rmse: 1.16567
[24]	valid_0's rmse: 1.16618
[25]	valid_0's rmse: 1.16544
[26]	valid_0's rmse: 1.1658
[27]	valid_0's rmse: 1.16997
[28]	valid_0's rmse: 1.16984
[29]	valid_0's rmse: 1.16977
[30]	valid_0's rmse: 1.17297
[31]	valid_0's rmse: 1.17015
[32]	valid_0's rmse: 1.16859
[33]	valid_0's rmse: 1.16

In [37]:
# Scores for cross val
for score in best_scores:
    print(score['valid_0']['rmse'])

1.16168675046895
1.0747303390202974
1.122922029787501
1.2447147945421524
1.1403464707277025


In [38]:
# Importance rank for first model in cross val models
imprtc_df = pd.DataFrame()
imprtc_df['feature'] = sample_train_X.drop('meter_reading_log1p', axis=1).columns   
imprtc_df['importance'] = models[0].feature_importances_
imprtc_df.sort_values('importance', ascending=False, inplace= True)
print(imprtc_df)
print(sample_train_X.columns)

                           feature  importance
0                      building_id          44
101                     meter_mean          15
5                  air_temperature          15
104                      meter_std          12
1                            meter           9
2                          site_id           3
98                            hour           3
97                       dayofweek           2
102                      meter_max           2
7                  dew_temperature           2
85   sea_level_pressure_mean_lag72           1
70       air_temperature_max_lag72           1
42      air_temperature_max_lag3_c           1
71       air_temperature_min_lag72           1
69      air_temperature_mean_lag72           1
77      dew_temperature_mean_lag72           1
81    precip_depth_1_hr_mean_lag72           1
28      precip_depth_1_hr_std_lag3           1
12                     s_radiation           1
3                      primary_use           1
100          